# Modeling Cross Section Data with KNN, DT, and Random Forest

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import os
from joblib import dump, load

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

In [36]:
import sys
# This allows us to import the nucml utilities
sys.path.append("../../..")

import nucml.exfor.data_utilities as exfor_utils
import nucml.endf.data_utilities as endf_utils
import nucml.plot.plotting_utilities as plot_utils
import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.model_building as model_building
import nucml.general_utilities as gen_utils

In [37]:
importlib.reload(exfor_utils)
importlib.reload(gen_utils)
importlib.reload(endf_utils)
importlib.reload(plot_utils)
importlib.reload(nuc_data)
importlib.reload(ace_utils)
importlib.reload(model_building)
print("Finish re-loading scripts.")

Finish re-loading scripts.


In [38]:
figure_dir = "../Figures/EXFOR_N/DT/"

# Loading Normalized One-Hot Encoded Training, Validation, and Testing Datasets

In [39]:
df = nuc_data.load_exfor()

INFO:root: MODE: neutrons
INFO:root: LOW ENERGY: False
INFO:root: LOG: False
INFO:root: BASIC: -1
INFO:root: SCALER: STANDARD
INFO:root:Reading data from C:\Users\Pedro\Desktop\ML_Nuclear_Data\EXFOR\CSV_Files\EXFOR_neutrons\EXFOR_neutrons_MF3_AME_no_RawNaN.csv
INFO:root:Data read into dataframe with shape: (4255409, 104)
INFO:root:Finished. Resulting dataset has shape (4255409, 104)


In [12]:
list(df.columns)

['Projectile',
 'Target_Metastable_State',
 'MT',
 'Product_Metastable_State',
 'EXFOR_Status',
 'Center_of_Mass_Flag',
 'Energy',
 'dEnergy',
 'Data',
 'dData',
 'ELV/HL',
 'dELV/HL',
 'I78',
 'Short_Reference',
 'EXFOR_Accession_Number',
 'EXFOR_SubAccession_Number',
 'EXFOR_Pointer',
 'Z',
 'Reaction_Notation',
 'Title',
 'Year',
 'Author',
 'Institute',
 'Date',
 'Reference',
 'Dataset_Number',
 'EXFOR_Entry',
 'Reference_Code',
 'Projectile_Z',
 'Projectile_A',
 'Projectile_N',
 'Isotope',
 'Element',
 'N',
 'A',
 'Element_Flag',
 'Nucleus_Radius',
 'Neutron_Nucleus_Radius_Ratio',
 'O',
 'Mass_Excess',
 'dMass_Excess',
 'Binding_Energy',
 'dBinding_Energy',
 'B_Decay_Energy',
 'dB_Decay_Energy',
 'Atomic_Mass_Micro',
 'dAtomic_Mass_Micro',
 'S(2n)',
 'dS(2n)',
 'S(2p)',
 'dS(2p)',
 'Q(a)',
 'dQ(a)',
 'Q(2B-)',
 'dQ(2B-)',
 'Q(ep)',
 'dQ(ep)',
 'Q(B-n)',
 'dQ(B-n)',
 'S(n)',
 'dS(n)',
 'S(p)',
 'dS(p)',
 'Q(4B-)',
 'dQ(4B-)',
 'Q(d,a)',
 'dQ(d,a)',
 'Q(p,a)',
 'dQ(p,a)',
 'Q(n,a)',

In [5]:
df, x_train, x_test, y_train, y_test, to_scale, scaler = nuc_data.load_exfor(log=True, basic=0, num=True, low_en=True)

C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\EXFOR_neutrons\EXFOR_neutrons_MF3_AME_no_NaNRaw.csv
Reading data into dataframe...
 MODE: neutrons
 LOW ENERGY: True
 LOG: True
 BASIC: 0
Data read into dataframe with shape:  (4184245, 10)
Dropping unnecessary features and one-hot encoding categorical columns...
Splitting dataset into training and testing...
Normalizing dataset...
Finished. Resulting dataset has shape  (4184245, 49) 
Training and Testing dataset shapes are (3765820, 48) and (418425, 48) respesctively.


In [32]:
dt_saving_dir = '../ML_Data/Models/DT/'

dump(scaler, open(os.path.join(dt_saving_dir, 'dt_scaler.pkl'), 'wb'))

# Sampling Chlorine (n,p) and U-233(n,f) Data for Visualization

The model will be trained in the entire data therefore getting the total MSE. As an example, we will see the model predictions for both the mentioned reactions. 

In [6]:
kwargs = {"nat_iso": "I", "one_hot": True, "scale": True, "scaler": scaler, "to_scale": to_scale}

chlorine_35_np = exfor_utils.load_exfor_samples(df, 17, 35, "MT_103", **kwargs)
uranium_235_nt = exfor_utils.load_exfor_samples(df, 92, 235, "MT_1", **kwargs)
uranium_233_nf = exfor_utils.load_exfor_samples(df, 92, 233, "MT_18", **kwargs)
uranium_233_nt = exfor_utils.load_exfor_samples(df, 92, 233, "MT_1", one_hot=True)

Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (215, 49)
Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (40108, 49)
Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (94567, 49)
Extracting samples from dataframe.
EXFOR extracted DataFrame has shape:  (33384, 49)


In [7]:
kwargs = {"nat_iso": "I", "one_hot": True, "scale": True, "scaler": scaler, "to_scale": to_scale}
uranium = exfor_utils.load_exfor_element(df, 92, **kwargs)

Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (468123, 49)


# Newly Measured Chlorine (n,p) Unseen Data 

These data points are not in the current EXFOR data package and will be used to test the new modeling capabilities.

In [8]:
new_cl_data_kwargs = {"Z":17, "A":35, "MT":"MT_103", "log":True, "scale":True, "scaler":scaler, "to_scale":to_scale}
new_cl_data = exfor_utils.load_exfor_newdata("../EXFOR/New_Data/Chlorine_Data/new_cl_np.csv", df, **new_cl_data_kwargs)
new_cl_data.head()

Extracting samples from dataframe.
EXFOR extracted DataFrame has shape:  (215, 49)
Scaling dataset...
Expanded Dataset has shape:  (12, 49)


,Energy,Data,Target_Protons,Target_Neutrons,Target_Mass_Number,MT_1,MT_102,MT_16,MT_17,MT_2,MT_3,MT_4,MT_101,MT_103,MT_104,MT_41,MT_9000,MT_105,MT_32,MT_51,MT_33,MT_107,MT_24,MT_155,MT_158,MT_159,MT_108,MT_29,MT_1108,MT_113,MT_106,MT_22,MT_1003,MT_9001,MT_28,MT_111,MT_203,MT_2103,MT_112,MT_37,MT_161,MT_152,MT_153,MT_18,MT_160,Frame_L,Frame_C,Target_Flag_I,Target_Flag_N
0,6.383815,-1.779892,-1.1485,-1.17161,-1.15875,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,6.383815,-1.707744,-1.1485,-1.17161,-1.15875,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,6.401401,-1.583359,-1.1485,-1.17161,-1.15875,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,6.401401,-1.590067,-1.1485,-1.17161,-1.15875,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,6.411620,-1.350665,-1.1485,-1.17161,-1.15875,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


# ENDF Evaluation Cross Section Data for Chlorine (n,p) Reaction

These data points will serve to plot the current ENDFb5 data and compare it to the newly measured points along with the predictions.

In [9]:
# ENDF EXTRACTION IMPLEMENTED INTO ANOTHER FUNCTION, NO NEED TO OBTAIN MANUALLY IN FUTURE VERSIONS

endf_cl = nuc_data.load_endf("Cl035", "MT103", log=True)
endf_u  = nuc_data.load_endf("U233", "MT018", log=True)
endf_cl_37 = nuc_data.load_endf("Cl037", "MT102", log=True)
endf_fe_56 = nuc_data.load_endf("Fe056", "MT002", log=True)

C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Cl035\endfb8.0\tables\xs\n-Cl035-MT103.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (8791, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\U233\endfb8.0\tables\xs\n-U233-MT018.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (15345, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Cl037\endfb8.0\tables\xs\n-Cl037-MT102.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (20121, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Fe056\endfb8.0\tables\xs\n-Fe056-MT002.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (46021, 2)


# Importing Error Metrics and Loading ACE Energy Arrays

In [10]:
# ACE EXTRACTION IMPLEMENTED INTO ANOTHER FUNCTION, NO NEED TO OBTAIN MANUALLY IN FUTURE VERSIONS

ace_u = ace_utils.get_energies("92233", ev=True, log=True)
ace_cl = ace_utils.get_energies("17035", ev=True, log=True)
ace_cl_37 = ace_utils.get_energies("17037", ev=True, log=True)
ace_fe_56 = ace_utils.get_energies("26056", ev=True, log=True)

# Decision Trees Training

In [14]:
importlib.reload(model_building)

<module 'nucml.model.model_building' from '..\\nucml\\model\\model_building.py'>

In [16]:
dt_parameters = {"max_depth_list":np.arange(20, 320, 20), "min_split_list":[2, 5, 10, 15], "min_leaf_list":[1, 3, 5, 7, 10]}

dt_results = model_building.train_dt(x_train, y_train, x_test, y_test, dt_parameters, 
                                     save_models=True, save_dir=dt_saving_dir)

Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 2; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 5; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 5; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 10; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 10; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 10; Min Samples Leaf = 5
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 20; Min Samples Split = 10; Min Samples Leaf = 7
    Calculating error metri

    Saving Model...
Training Decision Tree Regressor
Max Depth = 100; Min Samples Split = 15; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 100; Min Samples Split = 15; Min Samples Leaf = 5
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 100; Min Samples Split = 15; Min Samples Leaf = 7
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 100; Min Samples Split = 15; Min Samples Leaf = 10
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 120; Min Samples Split = 2; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 120; Min Samples Split = 5; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 120; Min Samples Split = 5; Min Samples Leaf = 3

    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 10; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 10; Min Samples Leaf = 5
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 10; Min Samples Leaf = 7
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 15; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 15; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Samples Split = 15; Min Samples Leaf = 5
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 200; Min Sampl

    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 2; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 5; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 5; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 10; Min Samples Leaf = 1
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 10; Min Samples Leaf = 3
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples Split = 10; Min Samples Leaf = 5
    Calculating error metrics...
    Saving Model...
Training Decision Tree Regressor
Max Depth = 300; Min Samples 

# Inspecting Decision Tree Training Parameters

In [17]:
import plotly.express as px

In [33]:
dt_results = pd.read_csv("../ML_Data/Models/DT/dt_results.csv").sort_values(by="max_depth")

In [ ]:
fig = px.scatter_3d(dt_results, x='max_depth', y='msl', z='train_mae', color='train_mae', 
                    title="Minimum Samples in Leaf - Train Dataset")
fig.show()
fig.write_image(os.path.join(figure_dir, "Parameter_Tuning/dt_msl_train.svg"))
fig.write_html(os.path.join(figure_disr, "Parameter_Tuning/dt_msl_train.html"))

fig = px.scatter_3d(dt_results, x='max_depth', y='msl', z='test_mae', color='test_mae', 
                    title="Minimum Samples in Leaf - Test Dataset")
fig.show()
fig.write_image(os.path.join(figure_dir, "Parameter_Tuning/dt_msl_test.svg"))
fig.write_html(os.path.join(figure_dir, "Parameter_Tuning/dt_msl_test.html"))

In [ ]:
fig = px.scatter_3d(dt_results, x='max_depth', y='mss', z='train_mae', color='train_mae',
                    title="Minimum Samples for Split - Train Dataset")
fig.show()
fig.write_image(os.path.join(figure_dir, "Parameter_Tuning/dt_mss_train.svg"))
fig.write_html(os.path.join(figure_dir, "Parameter_Tuning/dt_mss_train.html"))

fig = px.scatter_3d(dt_results, x='max_depth', y='mss', z='test_mae', color='test_mae',
                    title="Minimum Samples for Split - Test Dataset")
fig.show()
fig.write_image(os.path.join(figure_dir, "Parameter_Tuning/dt_mss_test.svg"))
fig.write_html(os.path.join(figure_dir, "Parameter_Tuning/dt_mss_test.html"))

# Visualizing Results of All Models in Chlorine and Uranium

In [ ]:
def generate_cl_u_gif(model_df, df, figure_dir):
    for i in model_df["model_path"]:
        # Loading already saved Model
        print(i)
        model = load(i) 
        figure_path_cl_35 = figure_dir + "Chlorine_35/chlorine_" + os.path.splitext(os.path.basename(i))[0] + ".png"
        figure_path_u_233 = figure_dir + "Uranium_233/uranium_" + os.path.splitext(os.path.basename(i))[0] + ".png"
        figure_path_cl_37 = figure_dir + "Chlorine_37/chlorine_" + os.path.splitext(os.path.basename(i))[0] + ".png"
        figure_path_fe_56 = figure_dir + "Iron_56/iron_" + os.path.splitext(os.path.basename(i))[0] + ".png"

        cl_kwargs =  {"Z":17, "A":35, "MT":"MT_103", "clf_type":None, "scaler":scaler, "to_scale":to_scale, 
                      "E_min":0, "E_max":1.5E7, "N":0, "e_array":ace_cl, "log":True,  
                      "error":True, "show":True, "save":True, "path":figure_path_cl_35, "render":False}
        results_cl = exfor_utils.predicting_nuclear_xs(df, clf=model, new_data=new_cl_data, endf=endf_cl, **cl_kwargs)

        u_kwargs =  {"Z":92, "A":233, "MT":"MT_18", "clf_type":None, "scaler":scaler, "to_scale":to_scale, 
                      "E_min":0, "E_max":1.5E7, "N":0, "e_array":ace_u, "log":True, 
                     "error":True, "show":True, "save":True, "path":figure_path_u_233, "render":False}
        results_u = exfor_utils.predicting_nuclear_xs(df, clf=model, endf=endf_u, **u_kwargs)
        
        cl_37_kwargs =  {"Z":17, "A":37, "MT":"MT_102", "clf_type":None, "scaler":scaler, "to_scale":to_scale, 
                      "E_min":0, "E_max":1.5E7, "N":0, "e_array":ace_cl_37, "log":True,  
                      "error":True, "show":True, "save":True, "path":figure_path_cl_37, "render":False}
        results_cl_37 = exfor_utils.predicting_nuclear_xs(df, clf=model, endf=endf_cl_37, **cl_37_kwargs)
        
        fe_56_kwargs =  {"Z":26, "A":56, "MT":"MT_2", "clf_type":None, "scaler":scaler, "to_scale":to_scale, 
                      "E_min":0, "E_max":1.5E7, "N":0, "e_array":ace_fe_56, "log":True, 
                     "error":True, "show":True, "save":True, "path":figure_path_fe_56, "render":False}
        results_fe = exfor_utils.predicting_nuclear_xs(df, clf=model, endf=endf_fe_56, **fe_56_kwargs)

    plot_utils.create_gif(figure_dir + "Chlorine_35/", ".png", "chlorine_dt.gif", duration=0.5)
    plot_utils.create_gif(figure_dir + "Uranium_233/", ".png", "uranium_dt.gif", duration=0.5)
    plot_utils.create_gif(figure_dir + "Chlorine_37/", ".png", "chlorine_37_dt.gif", duration=0.5)
    plot_utils.create_gif(figure_dir + "Iron_56/", ".png", "iron_dt.gif", duration=0.5)
    
generate_cl_u_gif(dt_results, df, figure_dir)

# Getting Best Models and Visualizing Results

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
path_to_best_train_model = dt_results[dt_results.train_mae == dt_results.train_mae.min()]["model_path"].values[0]
print("Loading model saved at: {}".format(path_to_best_train_model))
train_model = load(path_to_best_train_model) 

path_to_best_test_model = dt_results[dt_results.test_mae == dt_results.test_mae.min()]["model_path"].values[0]
print("Loading model saved at: {}".format(path_to_best_test_model))
test_model = load(path_to_best_test_model) 

In [ ]:
figure_path_cl = os.path.join(figure_dir, "Chlorine_35/chlorine_35_dt.svg")

cl_kwargs =  {"Z":17, "A":35, "MT":"MT_103", "clf_type":None, "scaler":scaler, "to_scale":to_scale, "html":True,
              "e_array":"ace", "log":True, "show":True, "render":True, "save":True, "path":figure_path_cl}
results_cl = exfor_utils.predicting_nuclear_xs_v2(df, clf=train_model, new_data=new_cl_data, get_endf=True, **cl_kwargs)

In [ ]:
fig = px.bar(results_cl["error_metrics"][~results_cl["error_metrics"]["id"].str.contains("NEW")], 
             x="id", y="mae", color='mae', template="simple_white", title="EXFOR Data: ML and ENDF Error")
fig.show()
fig.write_image(os.path.join(figure_dir, "Chlorine_35/cl_35_error.svg"))
fig.write_html(os.path.join(figure_dir, "Chlorine_35/cl_35_error.html"))

fig = px.bar(results_cl["error_metrics"][results_cl["error_metrics"]["id"].str.contains("NEW")], 
             x="id", y="mae", color='mae', template="simple_white", title="New Data: ML and ENDF Error")
fig.show()
fig.write_image(os.path.join(figure_dir, "Chlorine_35/cl_35_new_error.svg"))
fig.write_html(os.path.join(figure_dir, "Chlorine_35/cl_35_new_error.html"))

In [ ]:
figure_path_cl_37 = os.path.join(figure_dir, "Chlorine_37/chlorine_37_knn.svg")

cl_37_kwargs =  {"Z":17, "A":37, "MT":"MT_102", "clf_type":None, "scaler":scaler, "to_scale":to_scale, "html":True,
              "e_array":"ace", "log":True, "show":True, "render":True, "save":True, "path":figure_path_cl_37}
results_cl_37 = exfor_utils.predicting_nuclear_xs_v2(df, clf=train_model, get_endf=True, **cl_37_kwargs)

In [ ]:
fig = px.bar(results_cl_37["error_metrics"], x="id", y="mae", color='mae', template="simple_white", 
             title="EXFOR Data: ML and ENDF Error")
fig.show()
fig.write_image(os.path.join(figure_dir, "Chlorine_37/cl_37_error.svg"))
fig.write_html(os.path.join(figure_dir, "Chlorine_37/cl_37_error.html"))

In [ ]:
figure_path_u = os.path.join(figure_dir, "Uranium_233/uranium_233_dt.svg")

order = {
    "3":"endf", 
    "1":"exfor_ml_original", 
    "2":"exfor_ml", 
    "4":"exfor_new"}

u_kwargs =  {"Z":92, "A":233, "MT":"MT_18", "clf_type":None, "scaler":scaler, "to_scale":to_scale, "html":True,
              "e_array":"ace", "log":True, "show":True, "render":True, "save":True, "path":figure_path_u,
             "order_dict":order}
results_u = exfor_utils.predicting_nuclear_xs_v2(df, clf=train_model, get_endf=True, **u_kwargs)

In [ ]:
fig = px.bar(results_u["error_metrics"], x="id", y="mae", color='mae', template="simple_white", 
             title="EXFOR Data: ML and ENDF Error")
fig.show()
fig.write_image(os.path.join(figure_dir, "Uranium_233/u_233_error.svg"))
fig.write_html(os.path.join(figure_dir, "Uranium_233/u_233_error.html"))

In [ ]:
figure_path_fe = os.path.join(figure_dir, "Iron_56/iron_56_knn.svg")

fe_kwargs =  {"Z":26, "A":56, "MT":"MT_2", "clf_type":None, "scaler":scaler, "to_scale":to_scale, "html":True,
              "e_array":"ace", "log":True, "show":True, "render":True, "save":True, "path":figure_path_fe}
results_fe = exfor_utils.predicting_nuclear_xs_v2(df, clf=train_model, get_endf=True, **fe_kwargs)

In [ ]:
fig = px.bar(results_fe["error_metrics"], x="id", y="mae", color='mae', template="simple_white", 
             title="EXFOR Data: ML and ENDF Error")
fig.show()
fig.write_image(os.path.join(figure_dir, "Iron_56/fe_56_error.svg"))
fig.write_html(os.path.join(figure_dir, "Iron_56/fe_56_error.html"))